# 10. Sims
Source: <br>
1. '6. Weather 
2. '7. Matchups
3. Statcast Data for base rates
4. Models<br>

Description: This simulates games. Output includes player fantasy points and game scores <br>

To do: <br>
1. Output more stats for players <br>

In [1]:
contestKey=146153398

# Imports

In [2]:
import random
import numpy as np
import pandas as pd
import os
import xlrd
import math
import glob
import datetime
from datetime import date
import time 
import pickle

from functools import partial
from joblib import Parallel, delayed


import sys
sys.path.append(r"C:\Users\james\Documents\MLB\Code")

# import import_ipynb

from Classes import *
from simulation_functions_test import *

model_path = r"C:\Users\james\Documents\MLB\Code\Models"
baseball_path = r"C:\Users\james\Documents\MLB\Data2"
download_path = r"C:\Users\james\Downloads"

# import warnings
# warnings.simplefilter(action="ignore")

Code was last run on: 2023-06-29


In [3]:
# Today's Date
# YYYY-MM-DD (datetime)
todaysdate_dt = datetime.date.today()

# YYYY-MM-DD (string)
todaysdate_dash = str(todaysdate_dt)

# MM/DD/YYYY
todaysdate_slash = todaysdate_dash.split("-")
todaysdate_slash = todaysdate_slash[1] + "/" + todaysdate_slash[2] + "/" + todaysdate_slash[0]

# YYYYMMDD
todaysdate = todaysdate_dash.replace("-", "")

In [4]:
# Load in neural network stat model
filename = os.path.join(model_path, "model1_146146_100se.sav")          
model1 = pickle.load(open(filename, 'rb'))
# was using 18 before
filename = os.path.join(model_path, "model2_9999_100se.sav")          
model2 = pickle.load(open(filename, 'rb'))
filename = os.path.join(model_path, "model3_5858_100se.sav")          
model3 = pickle.load(open(filename, 'rb'))
filename = os.path.join(model_path, "model4_6644_100se.sav")          
model4 = pickle.load(open(filename, 'rb'))

In [5]:
# Team Map
team_map = pd.read_csv(os.path.join(baseball_path, "Utilities", "Team Map.csv"))[['BBREFTEAM', 'FANGRAPHSTEAM', 'VENUE_ID']]
team_map = team_map.set_index('BBREFTEAM')

In [6]:
# Lists
# If you just want to run certain matchups
late_list = ['SFG_NYY 03302023 0105PM ET.xlsx']


# List of games that don't work
broken_list = []

In [7]:
# Read in double play odds 
dp_rates = pd.read_csv(os.path.join(baseball_path, "Statcast Data", "double_play_rates.csv"))
dp_rates.set_index(["runner_1b", "runner_2b", "runner_3b"], inplace=True)

# Read in double play base of outs odds
dp_base_rates = pd.read_csv(os.path.join(baseball_path, "Statcast Data", "double_play_base_rates.csv"))
dp_base_rates.set_index(["runner_1b", "runner_2b", "runner_3b"], inplace=True)

In [8]:
# Read in base path advances 
advances = pd.read_csv(os.path.join(baseball_path, "Statcast Data", "advance_rates.csv"))
advances = advances.set_index("dp").to_dict()

In [9]:
# Pulls
filename = os.path.join(model_path, "nn_pull.sav")

# Load in model
pull = pickle.load(open(filename, 'rb'))

In [10]:
# Read in daily weather
def create_weather_df(date=date):
    filename = "Daily_Weather_" + str(date) + ".xlsx"
    weather_df = pd.read_excel(os.path.join(baseball_path, "8. Weather", "A. Swish Analytics", filename)) # just switched back to original
    # Set venue id as index
    weather_df = weather_df.set_index('venue_id')
    
    return weather_df

In [11]:
def create_matchup_list(contestKey, date=date, late_list=late_list, late=False):
    # Should move this out of this code and keep it as only functions
    # Load in today's DK salaries
    dk_name = "Salaries " + str(contestKey) + ".csv"

    # Clean DK salaries for merge
    DKSalaries = pd.read_csv(os.path.join(baseball_path, "2. Contests", "C. Salaries", dk_name), encoding='iso-8859-1')

    # Change Ohtani's code. He won't match otherwise.
    DKSalaries['ID'] = np.where(DKSalaries['Name'] == "Shohei Ohtani", 134045, DKSalaries['ID'])

    # This creates the folder where game sims will end up 
    directory = "Matchups" + str(contestKey)
    
    matchup_folder = "Matchups" + str(contestKey)
    matchup_path = os.path.join(baseball_path, "9. Matchups", "B. Matchups", matchup_folder)
    
    matchup_list = os.listdir(matchup_path)
    
    # If you're not just running the late ones, make a new directory
    if late == False:
        try:
            os.mkdir(os.path.join(baseball_path, "10. Sims", "A. Players", directory))
        except:
            pass
    # I separated these when I was adding scores separately (having them together would break if A8. Sims existed). Unnecessary going forward.
    if late == False:
        try:
            os.mkdir(os.path.join(baseball_path, "10. Sims", "B. Scores", directory))
        except:
            pass
    
    if late == True:
        matchup_list = late_list
        
    opener_list = list(DKSalaries[(DKSalaries['Roster Position'] == "SP") & (DKSalaries['Salary'] < 5000)]['Name'].unique())
    
    return DKSalaries, matchup_list, matchup_path, opener_list

# Simulations

In [12]:
# This merges sim results with DK Salaries then adds average and exposure
def create_matchup_df(matchup, DKSalaries, weather_df, date, contestKey, opener_list,
            team_map=team_map, pull=pull, model1=model1, model2=model2, model3=model3, model4=model4, dp_rates=dp_rates, dp_base_rates=dp_base_rates,
            advances=advances, num_sims=1000):
    
    matchup_folder = "Matchups" + str(contestKey)
    matchup_path = os.path.join(baseball_path, "9. Matchups", "B. Matchups", matchup_folder)
    
    all_scores_list=[]
    game_template = create_game(matchup, matchup_path, team_map, weather_df)
    output = Parallel(n_jobs=-1, verbose=0)(delayed(sim_matchup)(matchup=matchup, matchup_path=matchup_path, team_map=team_map, weather_df=weather_df, pull=pull, 
                                model1=model1, model2=model2, model3=model3, model4=model4, 
                                dp_rates=dp_rates, dp_base_rates=dp_base_rates, advances=advances, opener_list=opener_list, 
                                all_scores_list=all_scores_list, game_template=game_template) for sims in range(num_sims))
    
    player_output = []
    away_score = []
    home_score = []
    for game_sim in output:
        player_output.append(game_sim[0])
        away_score.append(game_sim[1])
        home_score.append(game_sim[2])
    
    
    
    player_sims = pd.concat(player_output, axis=1)
    
    score_df = pd.DataFrame(list(zip(away_score, home_score)), columns=['Away', 'Home'])

    
    cols=[]
    count=0
    for column in player_sims.columns:
        if column == "FP":
            cols.append(f'FP{count}')
            count+=1
            continue
        cols.append(column)
    player_sims.columns=cols
    player_sims = player_sims.loc[:,~player_sims.columns.duplicated()].copy() 
    
    # Merge all scores onto players
    player_sims = DKSalaries.merge(player_sims, on='ID', how='inner')
    
    return player_sims, score_df

In [13]:
def run_all(team_map=team_map, pull=pull, model1=model1, model2=model2, model3=model3, model4=model4, dp_rates=dp_rates, dp_base_rates=dp_base_rates,
            advances=advances, num_sims=1000, broken_list=broken_list, date=date, contestKey=None, late_list=late_list, late=False):
    
    directory = "Matchups" + str(contestKey)
    
    # Create path to Player Sims folder
    path = os.path.join(baseball_path, "10. Sims", "A. Players", directory)
    # Find all Excel files
    player_sim_files = glob.glob(os.path.join(path , "*.xlsx"))
    # Delete all if we're running the whole thing from scratch 
    # We'll just write over the ones we want to replace if we're using a late_list
    if late == False:
        for f in player_sim_files:
            os.remove(f)
        
    
    weather_df = create_weather_df(date)
    DKSalaries, matchup_list, matchup_path, opener_list = create_matchup_list(contestKey, date, late_list, late)
    
    # Define a custom sorting key function that extracts the numbers from characters 17-20 in each string (the game time)
    # Note: AM games can mess this up a bit
    def sort_key(string):
        return int(string[17:21])

    try:
        # Sort matchups on game time
        matchup_list.sort(key=sort_key)
    except:
        pass
        
    print(matchup_list)
    # This loops over matchups and creates matchups and then exports them to excel
    for matchup in matchup_list:
        if matchup.endswith(".xlsx"):
            print(matchup)

            player_sims, score_df = create_matchup_df(matchup, DKSalaries, weather_df, date, contestKey, opener_list,
                    team_map=team_map, pull=pull, model1=model1, model2=model2, model3=model3, model4=model4, dp_rates=dp_rates, dp_base_rates=dp_base_rates,
                    advances=advances, num_sims=1000)

            # Fill missings with 0s
            player_sims.fillna(0, inplace=True)

            player_sims.rename(columns={'Name_x': 'Name', 'order':'Roster Order'}, inplace=True)
            try:
                player_sims.drop(columns=['Unnamed: 0', 'Name_y'], inplace=True)
            except:
                player_sims.drop(columns=['Name_y'], inplace=True)
            player_sims['Roster Order'] = player_sims['Roster Order'].astype(int)

            # Set baseline export constraints
            player_sims['Min Exposure'] = 0.0
            # player_sims['Max Exposure'] = np.where(player_sims['Roster Position'] == "P", 1, 0.5)
            player_sims['Max Exposure'] = np.where(player_sims['Roster Position'] == "P", 0.5, 0.5)

            points_cols = [col for col in player_sims.columns if 'FP' in col]
            player_sims['AvgPointsPerGame'] = player_sims[points_cols].mean(axis=1)

            directory = "Matchups" + str(contestKey)
            player_sims.to_excel(os.path.join(baseball_path, "10. Sims", "A. Players", directory, matchup))
            
            # Game Scores
            score_df.to_excel(os.path.join(baseball_path, "10. Sims", "B. Scores", directory, matchup))
                                         
                                         
            score_df['home_win'] = np.where(score_df['Away'] < score_df['Home'], 1, 0)
            score_df['total'] = score_df['Away'] + score_df['Home']
            
            print("Home Win Rate: " + str(score_df['home_win'].mean()))
            print("Mean Away Score: " + str(score_df['Away'].mean()))
            print("Mean Home Score: " + str(score_df['Home'].mean()))
            print("Mean Total: " + str(score_df['total'].mean()))
            print("Median Total: " + str(score_df['total'].median()) + "\n")

In [14]:
# This appends all games together
def append_all(date=date, contestKey=None):  
    # Find all Excel files
    player_sim_files = glob.glob(os.path.join(baseball_path, "10. Sims", "A. Players", "Matchups" + str(contestKey), "*.xlsx"))

    # Append all together
    matchup_sim_list = []
    for filename in player_sim_files:
        df = pd.read_excel(filename, index_col=None, header=0)
        matchup_sim_list.append(df)

    all_matchup_sims = pd.concat(matchup_sim_list, axis=0, ignore_index=True)

    # Set file name
    sim_file = "Player Sims " + str(contestKey) + ".csv"

    # Sort
    all_matchup_sims.sort_values(['AvgPointsPerGame'], ascending=False, inplace=True)
    # Clean name
    all_matchup_sims = all_matchup_sims.loc[:,~all_matchup_sims.columns.str.startswith('Unnamed')]

    # Export to CSV
    all_matchup_sims.to_csv(os.path.join(baseball_path, "10. Sims", "A. Players", sim_file), encoding='iso-8859-1')

### Run One

In [15]:
%%time
run_all(date=todaysdate, contestKey=contestKey, late_list=['SDP_ARI 04202023 0940PM ET', 'NYM_SFG 04202023 0945PM ET'], num_sims=1000, late=False)
append_all(todaysdate, contestKey)

['SFG_TOR 06292023 0707PM ET.xlsx', 'MIL_NYM 06292023 0710PM ET.xlsx', 'HOU_STL 06292023 0715PM ET.xlsx', 'PHI_CHC 06292023 0805PM ET.xlsx', 'LAD_COL 06292023 0840PM ET.xlsx']
SFG_TOR 06292023 0707PM ET.xlsx
Home Win Rate: 0.4
Mean Away Score: 4.649
Mean Home Score: 3.692
Mean Total: 8.341
Median Total: 8.0

MIL_NYM 06292023 0710PM ET.xlsx
Home Win Rate: 0.547
Mean Away Score: 4.002
Mean Home Score: 4.346
Mean Total: 8.348
Median Total: 8.0

HOU_STL 06292023 0715PM ET.xlsx
Home Win Rate: 0.457
Mean Away Score: 5.141
Mean Home Score: 4.376
Mean Total: 9.517
Median Total: 9.0

PHI_CHC 06292023 0805PM ET.xlsx
Home Win Rate: 0.524
Mean Away Score: 4.518
Mean Home Score: 4.765
Mean Total: 9.283
Median Total: 9.0

LAD_COL 06292023 0840PM ET.xlsx
Home Win Rate: 0.411
Mean Away Score: 5.601
Mean Home Score: 4.252
Mean Total: 9.853
Median Total: 9.0

CPU times: total: 1min 1s
Wall time: 1min 25s


### Run All

In [16]:
# %%time
# # Read in history file        
# history = pd.read_csv(os.path.join(baseball_path, "Utilities", "draftkings-contest-entry-history.csv"))
# history['date'] = pd.to_datetime(history['Contest_Date_EST']).dt.strftime('%Y%m%d')
# history.drop_duplicates('Contest_Key', inplace=True)
# history = history[history['date'] > "20220301"]
# history = history[history['Sport'] == "MLB"]
# history = history[history['Contest_Entries'] > 8]
# history = history.reset_index(drop=True)


# for i in range(len(history)):
#     print("Contest: {}, Date: {}".format(history['Contest_Key'][i], history['date'][i]))
#     try:
#         run_all(date=history['date'][i], contestKey=history['Contest_Key'][i], late_list=['SDP_ARI 04202023 0940PM ET', 'NYM_SFG 04202023 0945PM ET'], num_sims=1000, late=False)
#         append_all(date=history['date'][i], contestKey=history['Contest_Key'][i])
#     except:
#         "Didn't work"

In [17]:
print("Code was last run on: {} at {}.".format(datetime.date.today(), datetime.datetime.now().strftime("%H:%M:%S")))

Code was last run on: 2023-06-29 at 18:25:39.
